In [3]:
# Setup
%matplotlib inline
%load_ext autoreload
%autoreload 2
import warnings
import spacy
from modified_anchor import anchor_text, anchor_base
import pickle
import myUtils
from myUtils import *
from transformer.utils import *
from dataset.dataset_loader import *
import datetime
%load_ext line_profiler

SEED = 84
torch.manual_seed(SEED)
warnings.simplefilter("ignore")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
# can be sentiment/spam/offensive
dataset_name = 'offensive'
text_parser, label_parser, ds_train, ds_val = get_dataset(dataset_name)

19190
4163
Number of tokens in training samples: 6226
Number of tokens in training labels: 2


In [5]:
len(ds_val)

2326

In [6]:
model = load_model('gru' , f'transformer/{dataset_name}/gru.pt', text_parser)
model = torch.jit.script(model)
myUtils.model = model
myUtils.text_parser = text_parser

{'embedding_dim': 100, 'batch_size': 32, 'hidden_dim': 256, 'num_layers': 2, 'dropout': 0.3, 'lr': 5e-05, 'early_stopping': 5, 'output_classes': 2}
VanillaGRU(
  (embedding_layer): Embedding(6226, 100)
  (GRU_layer): GRU(100, 256, num_layers=2, dropout=0.3)
  (dropout_layer): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=256, out_features=2, bias=True)
  (log_softmax): LogSoftmax(dim=1)
)


In [7]:
nlp = spacy.load('en_core_web_sm')

In [5]:
train, train_labels, test, test_labels, anchor_examples = preprocess_examples(ds_train)

In [6]:
ignored = get_ignored(anchor_examples)
normal_occurences = get_occurences(anchor_examples)
anchor_base.AnchorBaseBeam.best_group = BestGroup(normal_occurences)

## notice!

In [7]:
ignored = []

In [8]:
anchor_examples = anchor_examples[:1]

In [9]:
!nvidia-smi

Sat Jul  2 15:47:35 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.103.01   Driver Version: 470.103.01   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:82:00.0 Off |                  N/A |
| 45%   76C    P2    82W / 250W |   6809MiB / 11178MiB |     44%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
optimize = True
anchor_text.AnchorText.set_optimize(optimize)
explainer = anchor_text.AnchorText(nlp, ['positive', 'negative'], use_unk_distribution=False)

In [17]:
my_utils = TextUtils(anchor_examples, test, explainer, predict_sentences, ignored, f"profile.pickle", optimize=optimize)
myUtils.model = model
myUtils.text_parser = text_parser
set_seed()
%prun -s cumtime -T profile.txt my_utils.compute_explanations(list(range(len(anchor_examples))))

number 0
[0.948936170212766]
[0.9195402298850575]
[0.9337016574585635]
[0.9008264462809917]
[0.9803921568627451]
[0.8765432098765432]
[0.9083969465648855]
[1.0]
[0.9736842105263158]
[0.9083969465648855]
[0.9205298013245033]
[0.9245283018867925]
 
*** Profile printout saved to text file 'profile.txt'. 


         2152595 function calls (2152481 primitive calls) in 4.339 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    4.339    4.339 {built-in method builtins.exec}
        1    0.000    0.000    4.339    4.339 <string>:1(<module>)
        1    0.000    0.000    4.339    4.339 myUtils.py:170(compute_explanations)
        1    0.011    0.011    4.251    4.251 myUtils.py:147(get_exp)
        1    0.003    0.003    4.240    4.240 anchor_text.py:216(explain_instance)
        1    0.002    0.002    4.118    4.118 anchor_base.py:288(anchor_beam)
      158    0.031    0.000    3.987    0.025 anchor_text.py:174(sample_fn)
      154    0.001    0.000    3.895    0.025 anchor_base.py:241(<lambda>)
      154    0.015    0.000    3.894    0.025 anchor_base.py:187(complete_sample_fn)
     1508    0.037    0.000    3.254    0.002 anchor_text.py:97(sample)
     1520    0.018    0.000    2.547    0.002 anchor_

In [12]:
###### my_utils = TextUtils(anchor_examples, test, explainer, predict_sentences, ignored,f"profile.pickle")
#%lprun -s -m modified_anchor.anchor_text -m modified_anchor.anchor_base -m myUtils -T profile.txt  my_utils.compute_explanations(list(range(len(anchor_examples))))

In [13]:
print(datetime.datetime.now())

2022-07-02 15:47:46.806118
